In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import cv2
import os
import numpy as np
from skimage.feature import hog
from skimage import exposure
from google.colab.patches import cv2_imshow

# Define the target image size
target_size = (224, 224)

# normal data
image_normal_data = []
hog_features = []
labels = []
# Loop through all images in the dataset directory
for image_file in os.listdir("/content/drive/MyDrive/Normal cases"):
    # Read the image
    image = cv2.imread(os.path.join("/content/drive/MyDrive/Normal cases", image_file))

    lung_image = image.copy()

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(lung_image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to segment lung regions
    _, thresholded_image = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(thresholded_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Extract the largest contour (assumed to be the lungs)
    if contours:
        lung_contour = max(contours, key=cv2.contourArea)

        # Create a mask for the lung contour
        lung_mask = np.zeros_like(gray_image)
        cv2.drawContours(lung_mask, [lung_contour], -1, (255), thickness=cv2.FILLED)

        # Apply the mask to extract the lung ROI
        lung_roi = cv2.bitwise_and(gray_image, gray_image, mask=lung_mask)

        # Apply additional processing to enhance nodule detection (e.g., filtering, thresholding)
        # Here, you can perform processing steps to identify nodules within the lung ROI

        # Detect nodules by finding contours within the lung ROI
        nodule_contours, _ = cv2.findContours(lung_roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Draw contours around detected nodules on the original image
        cv2.drawContours(lung_image, nodule_contours, -1, (0, 0, 255), thickness=2)

        for nodule_contour in nodule_contours:
            x, y, w, h = cv2.boundingRect(nodule_contour)
            nodule = lung_roi[y:y + h, x:x + w]  # Extract the nodule region

            # Calculate HOG features for the nodule
            hog_feature = hog(nodule, orientations=9, pixels_per_cell=(8, 8),
                              cells_per_block=(2, 2), block_norm='L2-Hys')

            hog_features.append(hog_feature)
            labels.append(0)

        # Display the original image with contours around the detected nodules
        cv2_imshow(image)
        cv2_imshow(lung_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("No contours found in the image.")


    # Resize the image
    resized_image = cv2.resize(image, target_size)

    # Normalize the image pixel values
    normalized_image = resized_image / 255.0


    image_normal_data.append(normalized_image)
    # hog_feature = hog(resized_image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys')
    # hog_features.append(hog_feature)

X_normal = np.array(image_normal_data)
print(X_normal)



image_malignant_data = []  # Renamed to represent images from the 'Malignant cases' directory
hog_features = []
labels = []

# Loop through all images in the 'Malignant cases' directory
for image_file in os.listdir("/content/drive/MyDrive/Malignant cases"):  # Changed directory to 'Malignant cases'
    # Read the image
    image = cv2.imread(os.path.join("/content/drive/MyDrive/Malignant cases", image_file))  # Updated directory path

    lung_image = image.copy()

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(lung_image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to segment lung regions
    _, thresholded_image = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(thresholded_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Extract the largest contour (assumed to be the lungs)
    if contours:
        lung_contour = max(contours, key=cv2.contourArea)

        # Create a mask for the lung contour
        lung_mask = np.zeros_like(gray_image)
        cv2.drawContours(lung_mask, [lung_contour], -1, (255), thickness=cv2.FILLED)

        # Apply the mask to extract the lung ROI
        lung_roi = cv2.bitwise_and(gray_image, gray_image, mask=lung_mask)

        # Apply additional processing to enhance nodule detection (e.g., filtering, thresholding)
        # Here, you can perform processing steps to identify nodules within the lung ROI

        # Detect nodules by finding contours within the lung ROI
        nodule_contours, _ = cv2.findContours(lung_roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Draw contours around detected nodules on the original image
        cv2.drawContours(lung_image, nodule_contours, -1, (0, 0, 255), thickness=2)

        for nodule_contour in nodule_contours:
            x, y, w, h = cv2.boundingRect(nodule_contour)
            nodule = lung_roi[y:y + h, x:x + w]  # Extract the nodule region

            # Calculate HOG features for the nodule
            hog_feature = hog(nodule, orientations=9, pixels_per_cell=(8, 8),
                              cells_per_block=(2, 2), block_norm='L2-Hys')

            hog_features.append(hog_feature)
            labels.append(1)  # Change label to '1' for Malignant cases

        # Display the original image with contours around the detected nodules
        cv2_imshow(image)
        cv2_imshow(lung_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("No contours found in the image.")


    # Resize the image
    resized_image = cv2.resize(image, target_size)

    # Normalize the image pixel values
    normalized_image = resized_image / 255.0


    image_malignant_data.append(normalized_image)  # Updated list name to represent Malignant cases

X_malignant = np.array(image_malignant_data)  # Renamed variable to represent Malignant cases
print(X_malignant)




image_bengin_data = []
# Loop through all images in the dataset directory
for image_file in os.listdir("/content/drive/MyDrive/Bengin cases"):
    # Read the image
    image = cv2.imread(os.path.join("/content/drive/MyDrive/Bengin cases", image_file))
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    # Resize the image
    resized_image = cv2.resize(image, target_size)

    # Normalize the image pixel values
    normalized_image = resized_image / 255.0

    image_bengin_data.append(normalized_image)
    hog_feature = hog(resized_image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys')
    hog_features.append(hog_feature)

X_bengin = np.array(image_bengin_data)
X_bengin

hog_features

ModuleNotFoundError: No module named 'cv2'